Testing new visualisation features
---

---

In [1]:
from pathlib import Path
import os
_cwd = os.getcwd()

In [2]:
os.chdir(Path(_cwd)/'..') # .\SAE\
os.getcwd()

'C:\\Users\\Giulia\\Study\\projects\\research\\SAE'

In [11]:
import numpy as np
import torch
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
from models import GenerativeAE
import itertools

In [12]:
from experiments import EvaluationManager

In [13]:
import importlib
importlib.reload(EvaluationManager)

<module 'experiments.EvaluationManager' from 'C:\\Users\\Giulia\\Study\\projects\\research\\SAE\\experiments\\EvaluationManager.py'>

In [17]:
from metrics import score_disentanglement 
importlib.reload(score_disentanglement)

<module 'metrics.score_disentanglement' from 'C:\\Users\\Giulia\\Study\\projects\\research\\SAE\\metrics\\score_disentanglement.py'>

In [18]:
from experiments.EvaluationManager import ModelHandler

In [19]:
handler = ModelHandler(model_name="BaseSAE", model_version="v16", data="RFDh5")

Found existing config file: loading.
====== Opening RFD Dataset - train ======
Factors dictionary already created. Proceed to reading.
====== Opening RFD Dataset - test ======
Factors dictionary already created. Proceed to reading.
====== Opening RFD Dataset - HC ======
Factors dictionary already created. Proceed to reading.
====== Opening RFD Dataset - real ======
Categorising labels...
Factors dictionary already created. Proceed to reading.
BaseSAE model loaded.


In [20]:
checkpoints = handler.list_available_checkpoints()

Available checkpoints at logs\BaseSAE\v16_RFDh5\checkpoints :
['logs\\BaseSAE\\v16_RFDh5\\checkpoints\\epoch=0-step=1799-v1.ckpt', 'logs\\BaseSAE\\v16_RFDh5\\checkpoints\\epoch=0-step=1799.ckpt', 'logs\\BaseSAE\\v16_RFDh5\\checkpoints\\epoch=0-step=198.ckpt', 'logs\\BaseSAE\\v16_RFDh5\\checkpoints\\epoch=1-step=1998.ckpt']


In [21]:
handler.load_checkpoint()

Loading latest checkpoint at logs\BaseSAE\v16_RFDh5\checkpoints\epoch=1-step=1998.ckpt .
====== Opening RFD Dataset - train ======
Factors dictionary already created. Proceed to reading.
====== Opening RFD Dataset - test ======
Factors dictionary already created. Proceed to reading.
====== Opening RFD Dataset - HC ======
Factors dictionary already created. Proceed to reading.
====== Opening RFD Dataset - real ======
Categorising labels...
Factors dictionary already created. Proceed to reading.


In [22]:
scores = handler.score_model(FID=False, disentanglement=True)

Scoring model disentanglement.
DCI scoring
BetaVAE scoring


TypeError: expected Tensor as element 0 in argument 0, but got tuple